In [53]:
import os
import traceback

import pandas as pd
from certa.explain import CertaExplainer
from certa.local_explain import get_original_prediction, get_row
from certa.utils import merge_sources

from baselines.landmark import Landmark
from baselines.mojito import Mojito
import shap

import json
import warnings
warnings.filterwarnings("ignore")

In [4]:
root_datadir = '../cheapER/datasets/'
experiments_dir = '../examples/'
dataset = 'dirty_dblp_acm'
datadir = os.path.join(root_datadir, dataset)

In [7]:
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], [])
train_df = merge_sources(gt, 'ltable_', 'rtable_', lsource, rsource, ['label'], ['id'])

In [5]:
from models.utils import get_model
model_name = 'ditto'
save_path = 'models/saved/' + model_name + '/' + dataset
model = get_model(model_name, save_path, datadir, dataset)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

working on dirty_dblp_acm
reading data from ../cheapER/datasets/dirty_dblp_acm
data loaded
loading model from models/saved/ditto/dirty_dblp_acm


In [ ]:
p, r, f1 = model.evaluation(test_df)
print(f1)

In [29]:
def predict_fn(x):
    return model.predict(x)


In [68]:
certa_explainer = CertaExplainer(lsource, rsource)

In [8]:
predictions = model.predict(test_df)
predictions

,label,ltable_authors,ltable_title,ltable_venue,ltable_year,rtable_authors,rtable_title,rtable_venue,rtable_year,classes,labels,nomatch_score,match_score,ltable_id,rtable_id
0,0,NaN,secure transaction processing in firm real-tim...,NaN,1997.0,"binto george , jayant r. haritsa",secure buffering in firm real-time database sy...,the vldb journal -- the international journal ...,NaN,1,0,0.491374,0.508626,1496.0,1225.0
1,0,NaN,clustering validity checking methods : part ii...,NaN,2002.0,"maria halkidi , yannis batistakis , michalis v...",cluster validity methods : part i 2002,acm sigmod record,NaN,1,0,0.495292,0.504708,574.0,749.0
2,0,"gerd g. hillebrand , peter buneman , susan b. ...",a query language and optimization techniques f...,sigmod conference,1996.0,NaN,fundamental techniques for order optimization ...,NaN,1996.0,1,0,0.487596,0.512404,561.0,1957.0
3,0,NaN,structures for manipulating proposed updates i...,sigmod conference,NaN,won kim,observations on the odmg-93 proposal for an ob...,acm sigmod record,1994.0,0,0,0.503002,0.496998,1799.0,1307.0
4,0,NaN,integrating a structured-text retrieval system...,vldb,1994.0,NaN,index nesting - an efficient approach to index...,NaN,NaN,1,0,0.483793,0.516207,1116.0,1205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,1,NaN,discovery of influence sets in frequently upda...,NaN,2001.0,"ioana stanoi , mirek riedewald , divyakant agr...",discovery of influence sets in frequently upda...,NaN,2001.0,1,1,0.496201,0.503799,2202.0,1324.0
2469,0,"s. sudarshan , divesh srivastava , raghu ramak...",space optimization in deductive databases acm ...,NaN,NaN,NaN,some issues in design of distributed deductive...,very large data bases,NaN,1,0,0.476496,0.523504,94.0,2277.0
2470,0,NaN,efficient geometry-based similarity search of ...,NaN,1999.0,NaN,fast parallel similarity search in multimedia ...,NaN,1997.0,1,0,0.476933,0.523067,5.0,1921.0
2471,1,"viswanath poosala , phillip b. gibbons , yossi...",fast incremental maintenance of approximate hi...,NaN,2002.0,NaN,fast incremental maintenance of approximate hi...,acm transactions on database systems ( tods ),2002.0,1,1,0.494144,0.505856,2451.0,539.0


In [14]:
false_positives = predictions.loc[(predictions['label'] == 0) & (predictions['match_score'] > 0.5)]
false_negatives = predictions.loc[(predictions['label'] == 1) & (predictions['match_score'] < 0.5)]

In [103]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

In [ ]:
i = 0
rand_row =  false_negatives.iloc[i]
l_id = int(rand_row['ltable_id'])
label = rand_row["label"]
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]
rand_row.head()

item = get_row(l_tuple, r_tuple)
item.drop(['ltable_id','rtable_id'])

In [143]:
original_prediction = false_negatives.iloc[i][['nomatch_score', 'match_score']]
original_prediction

nomatch_score    0.50709
match_score      0.49291
Name: 149, dtype: object

In [144]:
score = original_prediction[1]
score

0.49291

In [ ]:
label = false_positives.iloc[i]['label']
label

In [ ]:
pd.DataFrame(data=[label, score], columns=['label','score'])

In [146]:
saliency_df, cf_summary, counterfactual_examples, triangles = certa_explainer.explain(l_tuple, r_tuple, predict_fn, num_triangles=10)

In [177]:
saliency_df.style.background_gradient(cmap=cm, axis=1, low=0.2)

,ltable_title,ltable_authors,ltable_venue,ltable_year,rtable_title,rtable_authors,rtable_venue,rtable_year
0,0.533898,0.491525,0.500000,0.491525,0.203390,0.186441,0.186441,0.186441


In [166]:
landmark_explainer = Landmark(lambda x: predict_fn(x)['match_score'].values, test_df,
                              lprefix='',
                              exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
                              rprefix='',
                              split_expression=r' ')
labelled_item = item.copy()
labelled_item['label'] = int(label)
labelled_item['id'] = i
land_explanation = landmark_explainer.explain(labelled_item)
land_exp = land_explanation.groupby('column')['impact'].sum().to_dict()

In [167]:
pd.DataFrame(data=[land_exp.values()], columns=land_exp.keys()).style.background_gradient(cmap=cm, axis=1, low=0.2)

,ltable_authors,ltable_title,ltable_venue,ltable_year,rtable_authors,rtable_title,rtable_venue,rtable_year
0,0.001285,-0.001959,0.000028,-0.003825,-0.003141,0.002299,-0.002410,-0.000666


In [ ]:
shap_explainer = shap.KernelExplainer(lambda x: predict_fn(x)['match_score'].values,
                                      train_df.drop(['label'], axis=1).astype(str)[:50],
                                      link='identity')
shap_instance = test_df.iloc[i, 1:].drop(['ltable_id', 'rtable_id']).astype(str)
shap_values = shap_explainer.shap_values(shap_instance, nsamples=100)
match_shap_values = shap_values
shap_saliency = dict()
for sv in range(len(match_shap_values)):
    shap_saliency[train_df.columns[1 + sv]] = match_shap_values[sv]


In [ ]:
shap_saliency.style.background_gradient(cmap=cm, axis=1, low=0.2)

In [ ]:
mojito = Mojito(test_df.columns,
                attr_to_copy='left',
                split_expression=" ",
                class_names=['no_match', 'match'],
                lprefix='', rprefix='',
                feature_selection='lasso_path')
mojito_exp = mojito.drop(predict_fn_mojito, item,
                              num_features=15,
                              num_perturbation=100)
mojito_exp = mojito_exp.groupby('attribute')['weight'].mean().to_dict()
if 'id' in mojito_exp:
    mojito_exp.pop('id', None)
if 'ltable_id' in mojito_exp:
    mojito_exp.pop('ltable_id', None)
if 'rtable_id' in mojito_exp:
    mojito_exp.pop('rtable_id', None)

In [ ]:
mojito_exp.style.background_gradient(cmap=cm, axis=1, low=0.2)

In [ ]:
saliencies = []
saliencies.append(certa_saliency.copy())
saliencies.append(mojito_saliency.copy())
saliencies.append(landmark_saliency.copy())
saliencies.append(shap_saliency.copy())

lprefix = 'ltable_'
rprefix = 'rtable_'

k = 1
eval_series = pd.Series()
# eval saliencies
for saliency in saliencies:
    print(saliency)
    # get top k important attributes
    exp_type = saliency['type']
    print(saliency.pop('type'))
    if exp_type == 'certa':
        explanation_attributes = sorted(saliency, key=saliency.get, reverse=True)[:k]
    elif score < 0.5:
        saliency = {k:v for k,v in saliency.items() if v < 0}
        explanation_attributes = sorted(saliency, key=saliency.get)[:k]
    else:
        saliency = {k:v for k,v in saliency.items() if v > 0}
        explanation_attributes = sorted(saliency, key=saliency.get, reverse=True)[:k]

    # change those attributes
    try:
        lt = l_tuple.copy()
        rt = r_tuple.copy()
        modified_row = get_row(lt, rt)
        for e in explanation_attributes:
            modified_row[e] = ''
        modified_tuple_prediction = \
            predict_fn(modified_row)[['nomatch_score', 'match_score']].values[0]
        #print(modified_tuple_prediction)
        #print(modified_row)
        score_drop = modified_tuple_prediction[1]
        eval_series['top_' + exp_type + '_' + model.name] = explanation_attributes
        eval_series['score_drop_' + exp_type + '_' + model.name] = score_drop
    except Exception as e:
        print(traceback.format_exc())
    try:
        lt = l_tuple.copy()
        rt = r_tuple.copy()
        modified_row = get_row(lt, rt)
        for e in explanation_attributes:
            if e.startswith(lprefix):
                new_e = e.replace(lprefix, rprefix)
            else:
                new_e = e.replace(rprefix, lprefix)
            modified_row[e] = modified_row[new_e]
        #print(modified_row)
        modified_tuple_prediction = \
            predict_fn(modified_row)[['nomatch_score', 'match_score']].values[0]
        #print(modified_tuple_prediction)
        score_copy = modified_tuple_prediction[1]
        eval_series['score_copy_' + exp_type + '_' + model.name] = score_copy
    except Exception as e:
        print(traceback.format_exc())
print(eval_series)
print(cf_summary)

In [ ]:
import numpy as np
attributes = cf_summary.keys()[0].split('/')
print(attributes)
ex = counterfactual_examples[counterfactual_examples['alteredAttributes'].apply(lambda x: np.all([*map(lambda l: l in x, attributes)]))]
ex

In [ ]:
predict_fn(pd.DataFrame(ex))